In [20]:
import tensorflow as tf
# pip install tfds-nightly==4.9.2
import tensorflow_datasets as tfds

In [7]:
# 1. Load dataset
(train_ds, train_labels), (test_ds, test_labels) = tfds.load("tf_flowers",
                                                            split=["train[:70%]", "train[:30%]"],
                                                            batch_size=-1,
                                                            as_supervised=True)

In [9]:
# 2. resize images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

In [12]:
# 3. Proper format for labels, change categorical data to numerical
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [15]:
# 4. load pre trained vgg16 model
from tensorflow.keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=train_ds[0].shape)
# model that is pretrained on imagenet
# does not include the top classification layer

In [17]:
## 5. will not train base mode
# Freeze Parameters in model's lower convolutional layersb
base_model.trainable = False

In [18]:
# 6. preprocess input 
from tensorflow.keras.applications.vgg16 import preprocess_input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

In [23]:
# 7. Build model
from tensorflow.keras import models, layers
model = models.Sequential([
    base_model, #pretrained model
    layers.Flatten(), #flatten layer
    layers.Dense(50, activation='relu'), #dense layer 1
    layers.Dense(20, activation='relu'), #dense layer 2
    layers.Dense(5, activation='softmax'), #prediction layer
])

In [24]:
# 8. Compile 
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [27]:
# 9. Fit model with early stopping
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 5, restore_best_weights = True)

history = model.fit(train_ds, train_labels, epochs = 3, validation_split = 0.2, batch_size = 2, callbacks = [es])

Epoch 1/3
1028/1028 [==============================] - 237s 229ms/step - loss: 1.6785 - accuracy: 0.5061 - val_loss: 1.1618 - val_accuracy: 0.5447
Epoch 2/3
1028/1028 [==============================] - 238s 231ms/step - loss: 0.9714 - accuracy: 0.6501 - val_loss: 1.4042 - val_accuracy: 0.5778
Epoch 3/3
1028/1028 [==============================] - 238s 231ms/step - loss: 0.7948 - accuracy: 0.7328 - val_loss: 1.1093 - val_accuracy: 0.6420


In [29]:
loss, accuracy = model.evaluate(test_ds, test_labels)
print("loss: ", loss)
print("acc: ", accuracy)

35/35 [==============================] - 79s 2s/step - loss: 0.5500 - accuracy: 0.7947
loss:  0.5499953031539917
acc:  0.7947320342063904


In [31]:
import numpy as np

y_pred = model.predict(test_ds)
predicted = np.argmax(y_pred, axis=1)

print(predicted[:10])

actual = np.argmax(test_labels, axis=1)
print(actual[:10])

35/35 [==============================] - 77s 2s/step
[2 3 0 0 3 3 0 0 0 4]
[2 3 3 4 3 0 0 0 0 1]
